# <a id='in'>INDIA ANALYSIS </a>
With around 14,64,000 cases India ranks third in World for no of cases. With such a large population density and Second largest Population let's see how India Fares with other countries and how succesful are the lockdowns.

In [ ]:
import datetime as dt
curr_time = dt.datetime.now()
print('Last Updated: {} UTC'.format(curr_time.strftime("%Y-%m-%d %H:%M")))
print('Run the notebook again to get the latest figures.')

In [ ]:
!pip install pycountry_convert

In [ ]:
!pip install GoogleMaps

In [ ]:
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import googlemaps
import re 
import pycountry
import pycountry_convert as pc
import requests

In [ ]:
class country_utils():
    def __init__(self):
        self.d = {}
    
    def get_dic(self):
        return self.d
    
    def get_country_details(self,country):
        """Returns country code(alpha_3) and continent"""
        try:
            country_obj = pycountry.countries.get(name=country)
            if country_obj is None:
                c = pycountry.countries.search_fuzzy(country)
                country_obj = c[0]
            continent_code = pc.country_alpha2_to_continent_code(country_obj.alpha_2)
            continent = pc.convert_continent_code_to_continent_name(continent_code)
            return country_obj.alpha_3, continent
        except:
            if 'Congo' in country:
                country = 'Congo'
            if country == 'Mainland China':
                country = 'China'
            elif country == 'Diamond Princess' or country == 'Laos' or country == 'MS Zaandam'\
            or country == 'Holy See' or country == 'Timor-Leste':
                return country, country
            elif country == 'Korea, South' or country == 'South Korea':
                country = 'Korea, Republic of'
            elif country == 'Taiwan*':
                country = 'Taiwan'
            elif country == 'Burma':
                country = 'Myanmar'
            elif country == 'West Bank and Gaza':
                country = 'Gaza'
            else:
                return country, country
            country_obj = pycountry.countries.search_fuzzy(country)
            continent_code = pc.country_alpha2_to_continent_code(country_obj[0].alpha_2)
            continent = pc.convert_continent_code_to_continent_name(continent_code)
            return country_obj[0].alpha_3, continent
    
    def get_iso3(self, country):
        return self.d[country]['code']
    
    def get_continent(self,country):
        return self.d[country]['continent']
    
    def add_values(self,country):
        self.d[country] = {}
        self.d[country]['code'],self.d[country]['continent'] = self.get_country_details(country)
    
    def fetch_iso3(self,country):
        if country in self.d.keys():
            return self.get_iso3(country)
        else:
            self.add_values(country)
            return self.get_iso3(country)
        
    def fetch_continent(self,country):
        if country in self.d.keys():
            return self.get_continent(country)
        else:
            self.add_values(country)
            return self.get_continent(country)

In [ ]:
df_world = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')
df_world.ObservationDate = pd.to_datetime(df_world.ObservationDate, format='%m/%d/%Y')
max_date = df_world.ObservationDate.max()
df_world = df_world[df_world.ObservationDate==max_date]
df_world.rename(columns={'Country/Region':'Country'},inplace=True)
df_cont = df_world.copy()
df_world = df_world.groupby(['Country'],as_index=False)['Confirmed','Deaths','Recovered'].sum()
df_world['rank_c'] = df_world['Confirmed'].rank(ascending=False)
df_world['rank_d'] = df_world['Deaths'].rank(ascending=False)
df_world['rank_r'] = df_world['Recovered'].rank(ascending=False)
world_stat = (df_world.loc[df_world['Country']=='India'])
world_stat.set_index('Country',inplace=True)
world_stat = world_stat.astype(int)

In [ ]:
obj = country_utils()
df_cont['continent'] = df_cont.apply(lambda x: obj.fetch_continent(x['Country']), axis=1)
df_cont = df_cont.groupby(['continent','Country'],as_index=False)['Confirmed','Deaths','Recovered'].sum()
df_cont = df_cont[df_cont['continent']=='Asia']
df_cont['rank_c'] = df_cont['Confirmed'].rank(ascending=False)
df_cont['rank_d'] = df_cont['Deaths'].rank(ascending=False)
df_cont['rank_r'] = df_cont['Recovered'].rank(ascending=False)
cont_stat = (df_cont.loc[df_cont['Country']=='India'])
cont_stat.set_index('Country',inplace=True)
cont_stat.drop('continent',inplace=True,axis=1)
cont_stat = cont_stat.astype(int)

The table below shows the current figures and ranking for India in the World and in Asia.

In [ ]:
import plotly.graph_objects as go

values = [['Figures','World Ranking','Asia Ranking'],
          [world_stat.Confirmed['India'],world_stat.rank_c['India'],cont_stat.rank_c['India']],
          [world_stat.Deaths['India'],world_stat.rank_d['India'],cont_stat.rank_d['India']],
          [world_stat.Recovered['India'],world_stat.rank_r['India'],cont_stat.rank_r['India']]]


fig = go.Figure(data=[go.Table(
  columnorder = [1,2,3,4],
  columnwidth = [300,400],
  header = dict(
    values = [['<b>STATISTICS</b><br>as of '+str(max_date.day)+' '+ max_date.month_name()],
              ['<b>CASES</b>'],['<b>DEATHS</b>'],['<b>RECOVERIES</b>']],
    line_color='darkslategray',
    fill_color='royalblue',
    align=['left','center'],
    font=dict(color='white', size=12),
    height=40
  ),
  cells=dict(
    values=values,
    line_color='darkslategray',
    fill=dict(color=['paleturquoise', 'white']),
    align=['left', 'center'],
    font_size=12,
    height=30)
    )
])
fig.update_layout(height=250,title='India',template='presentation',margin=dict(l=20, r=20, t=50, b=20))
fig.show()

In [ ]:
df = pd.read_csv('/kaggle/input/covid19-in-india/covid_19_india.csv')
df['Date'] = pd.to_datetime(df['Date'],format='%d/%m/%y')
df['Date'] = df['Date'].dt.date
df.rename(columns={'Date':'date','State/UnionTerritory':'state','ConfirmedIndianNational':'confirmed_in',\
                   'ConfirmedForeignNational':'confirmed_fr'}, inplace=True)
df.drop(['Sno','Time'],axis=1,inplace=True)
df['state'] = df.apply(lambda x: 'Nagaland' if x['state']=='Nagaland#' else 'Jharkhand' if x['state']=='Jharkhand#' else x['state'], axis=1)
df = df[df['state']!='Unassigned']
df.reset_index(inplace=True)

In [ ]:
def add_daily_measures(df):
    has_state=False
    if 'state' in df.columns:
        states = []
        has_state = True
    df.loc[0,'Daily Cases'] = df.loc[0,'Confirmed']
    df.loc[0,'Daily Deaths'] = df.loc[0,'Deaths']
    df.loc[0,'Daily Cured'] = df.loc[0,'Cured']
    for i in range(1,len(df)):
        if has_state:
            if df.loc[i,'state'] in states:
                df.loc[i,'Daily Cases'] = df.loc[i,'Confirmed'] - df.loc[i-1,'Confirmed']
                df.loc[i,'Daily Deaths'] = df.loc[i,'Deaths'] - df.loc[i-1,'Deaths'] 
                df.loc[i,'Daily Cured'] = df.loc[i,'Cured'] - df.loc[i-1,'Cured']
            else:
                states.append(df.loc[i,'state'])
                df.loc[i,'Daily Cases'] = df.loc[i,'Confirmed']
                df.loc[i,'Daily Deaths'] = df.loc[i,'Deaths']
                df.loc[i,'Daily Cured'] = df.loc[i,'Cured']
        else:
            df.loc[i,'Daily Cases'] = df.loc[i,'Confirmed'] - df.loc[i-1,'Confirmed']
            df.loc[i,'Daily Deaths'] = df.loc[i,'Deaths'] - df.loc[i-1,'Deaths'] 
            df.loc[i,'Daily Cured'] = df.loc[i,'Cured'] - df.loc[i-1,'Cured']
    #Make the first row as 0 because we don't know the previous value
    df.loc[0,'Daily Cases'] = 0
    df.loc[0,'Daily Deaths'] = 0
    df.loc[0,'Daily Cured'] = 0
    return df

The plot below shows the total cases, deaths and recovered, Even after 4 lockdowns the cases continue to increase exponentially

In [ ]:
df.loc[1428,'state'] = 'Madhya Pradesh'
df.loc[1428,'Deaths'] = '119'
df.fillna(0,inplace=True)
df.loc[df.Deaths=='0#','Deaths'] = 0
df.Deaths = df.Deaths.astype(np.int16)

In [ ]:
df_ind = df.copy()
df_ind = df_ind.groupby('date',as_index=False)['Cured','Deaths','Confirmed'].sum()
df_ind = add_daily_measures(df_ind)
fig = go.Figure(data=[
    go.Bar(name='Deaths', x=df_ind['date'], y=df_ind['Daily Deaths'],marker_color='crimson',marker_line_color='black'),
    go.Bar(name='Recoveries', x=df_ind['date'], y=df_ind['Daily Cured'],marker_color='limegreen',marker_line_color='black'),
    go.Bar(name='Cases', x=df_ind['date'], y=df_ind['Daily Cases'],marker_color='royalblue',marker_line_color='black'),
])
# Change the bar mode
fig.update_layout(template="simple_white",barmode='relative', title='Total Cases, Deaths & Recoveries',
                 annotations=[dict(x='2020-03-23', y=73, xref="x", yref="y", text="First Lockdown(23rd March)", showarrow=True, arrowhead=1, ax=-100, ay=-100),
                              dict(x='2020-04-15', y=1290, xref="x", yref="y", text="Second Lockdown(15th April)", showarrow=True, arrowhead=1, ax=-100, ay=-100),
                              dict(x='2020-05-04', y=3670, xref="x", yref="y", text="Third Lockdown(4th May)", showarrow=True, arrowhead=1, ax=-100, ay=-100),
                              dict(x='2020-05-18', y=7994, xref="x", yref="y", text="Fourth Lockdown(18th May)", showarrow=True, arrowhead=1, ax=-100, ay=-100)
                             ])
fig.show()

In [ ]:
%%HTML
<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/2061549" data-url="https://flo.uri.sh/visualisation/2061549/embed"><script src="https://public.flourish.studio/resources/embed.js"></script></div>

In [ ]:

df_br = df.copy()
df_br = df_br.pivot(index='state',columns='date',values='Confirmed')
df_br.fillna(0,inplace=True)
df_br.reset_index(level=0, inplace=True)
df_br.columns.name = ''
df_br.to_csv(r'confirmed_cases_india.csv',index=False)

In [ ]:
def doubling_rate_india(x):
    dr = np.log(2)/np.log(x['Confirmed']/x['Confirmed_prev'])
    return np.round(dr,2)

df_ind['Confirmed_prev'] = df_ind['Confirmed'].shift(1)
df_ind['Doubling rate'] = df_ind.apply(lambda x: doubling_rate_india(x),axis=1)
fig = go.Figure()
fig.add_trace(go.Bar(x=df_ind['date'],y=df_ind['Doubling rate']))
fig.update_layout(template='presentation',title="India's Doubling rate variation")
fig.show()

In [ ]:
doubling_rate = 0
state = df.state.unique().tolist()[0]
def calc_doubling_rate(x):
    global growth_rate
    global state
    if x['state']!=state:
        doubling_rate=0
        state = x['state']
    try:
        dr = np.log(2)/np.log(x['Confirmed']/x['Confirmed_prev'])
    except ZeroDivisionError:
        dr = 0
    return np.round(dr,2)

df_dr = df.copy()
df_dr.sort_values(['state','date'],inplace=True)
df_dr.reset_index(drop=True,inplace=True)
df_dr['Confirmed_prev'] = df_dr.groupby('state')['Confirmed'].transform(lambda x: x.shift(1))
df_dr['Doubling rate'] = df_dr.apply(lambda x: calc_doubling_rate(x),axis=1)
df_curr_dr = df_dr[df_dr['date']==df_dr.date.max()]
df_curr_dr.sort_values('Confirmed',ascending=False,inplace=True)
df_curr_dr=df_curr_dr[['state','Confirmed','Doubling rate']]
df_curr_dr=df_curr_dr.nlargest(15,'Confirmed')
df_curr_dr.reset_index(drop=True,inplace=True)

# Set CSS properties for th elements in dataframe
th_props = [
  ('font-size', '11px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', '#6d6d6d'),
  ('background-color', '#f7f7f9')
  ]

# Set CSS properties for td elements in dataframe
td_props = [
  ('font-size', '11px')
  ]

# Set table styles
styles = [
  dict(selector="th", props=th_props),
  dict(selector="td", props=td_props)
  ]

(df_curr_dr.style
 .background_gradient(cmap='Reds_r',subset='Doubling rate')
 .background_gradient(cmap='Blues',subset='Confirmed')
 .set_caption('Doubling Rates of 15 Indian States with most cases')
 .set_table_styles(styles))

In [ ]:
df_dr = df_dr.merge(df_curr_dr['state'],how='inner',on='state')
rows=5
cols=3
states = df_curr_dr.state.tolist()
fig = make_subplots(rows,cols,shared_xaxes=True,subplot_titles=states)
for r in range(1,rows+1):
    for c in range(1,cols+1):
        ind = r+c-2
        df_s = df_dr[df_dr['state']==states[r+c-2]]
        fig.add_trace(go.Bar(x=df_s['date'],y=np.abs(df_s['Doubling rate'])),r,c)
fig.update_layout(template='plotly_white',showlegend=False,title='State-wise Doubling Rate variation')
fig.show()

In [ ]:
fig = px.bar(df_dr,x='date',y='Doubling rate',color='state')
fig.update_layout(title='Stacked bar plot representing doubling rates',template='plotly_white')
fig.show()

In [ ]:
def change_state_name(state):
    if state == 'Odisha':
        return 'Orissa'
    elif state == 'Telengana':
        return 'Telangana'
    elif state == 'Andaman and Nicobar Islands':
        return 'Andaman and Nicobar'
    return state
r = requests.get(url='https://raw.githubusercontent.com/geohacker/india/master/state/india_telengana.geojson')
geojson = r.json()
df_map['state'] = df_map.apply(lambda x: change_state_name(x['state']), axis=1)
last_date = df_map.date.max()
df_map = df_map[df_map['date']==last_date]
fig = px.choropleth_mapbox(df_map, geojson=geojson, color="Confirmed",
                    locations="state", featureidkey="properties.NAME_1",
                    hover_name='state',
                    hover_data=['Cured','Deaths'],
                    center={"lat": 20.5937, "lon": 78.9629},
                    mapbox_style="carto-positron",
                    zoom=2.75,
                    color_continuous_scale=px.colors.qualitative.Vivid,
                    title='Total Cases per State'
                   )
fig.update_geos(fitbounds="locations", visible=True)
fig.update_geos(projection_type="orthographic")
fig.show()

In [ ]:
df_ind_det = pd.read_csv('/kaggle/input/covid19-in-india/IndividualDetails.csv')
r = requests.get(url='https://raw.githubusercontent.com/geohacker/india/master/district/india_district.geojson')
geojson = r.json()
df_dist = df_ind_det.groupby('detected_district',as_index=False)['id'].count()
df_dist.rename(columns={'detected_district':'District','id':'Cases Reported'},inplace=True)
fig = px.choropleth_mapbox(df_dist, geojson=geojson, color="Cases Reported",
                    locations="District", featureidkey="properties.NAME_2",
                    hover_name='District',
                    center={"lat": 20.5937, "lon": 78.9629},
                    mapbox_style="carto-positron",
                    zoom=2.75,
                    color_continuous_scale=px.colors.qualitative.Vivid,
                    title='Total Cases per District'
                   )
fig.update_geos(fitbounds="locations", visible=True)
fig.update_geos(projection_type="orthographic")
fig.show()

In [ ]:
df_zone = pd.read_csv('/kaggle/input/covid19indiazones/India-District-Zones.csv')
df_zone = df_zone.groupby(['State','Zone'],as_index=False)['District'].count()
df_zone.sort_values('District',inplace=True)
fig = go.Figure()
df_g = df_zone[df_zone['Zone']=='Green Zone']
fig.add_trace(go.Bar(x=df_g['State'],y=df_g['District'], marker_color='Green'))
df_o = df_zone[df_zone['Zone']=='Orange Zone']
fig.add_trace(go.Bar(x=df_o['State'],y=df_o['District'], marker_color='Orange'))
df_r = df_zone[df_zone['Zone']=='Red Zone']
fig.add_trace(go.Bar(x=df_r['State'],y=df_r['District'], marker_color='Red'))
fig.update_layout(title='Number of Zones per State from 4th May',showlegend=False, barmode='stack')
fig.show()

## Testing Details

In [ ]:
df_lab_group = df_lab.groupby('state',as_index=False)['lab'].count().sort_values('lab',ascending=False)
fig = px.bar(df_lab_group,x='state',y='lab',title='Number of COVID19 testing labs per State/Union Territory')
fig.update_layout(template='plotly_white')
fig.update_yaxes(title_text = 'Number of Labs')
fig.update_xaxes(title_text = '')
fig.show()

In [ ]:
df_tes_st = pd.read_csv('/kaggle/input/covid19-in-india/StatewiseTestingDetails.csv')
fig = go.Figure()
for state in df_tes_st.State.unique():
    df_s = df_tes_st[df_tes_st['State']==state]
    fig.add_trace(go.Scatter(x=df_s['Date'],y=df_s['TotalSamples'],mode='lines+markers',name=state))
    #fig.add_trace(go.Bar(x=df_s['Date'],y=df_s['Positive']))
fig.update_layout(template='plotly_white',title='State-wise number of tests',xaxis=dict(range=[df_tes_st.Date.min(),df.date.max()]))
fig.show()

#  Statewise Analysis

In [ ]:
df_pop = pd.read_csv('/kaggle/input/covid19-in-india/population_india_census2011.csv')
df_bed = pd.read_csv('/kaggle/input/covid19-in-india/HospitalBedsIndia.csv')
df_test = pd.read_csv('/kaggle/input/covid19-in-india/StatewiseTestingDetails.csv',parse_dates=True)
df_bed['Total beds'] = df_bed.NumPublicBeds_HMIS + df_bed.NumRuralBeds_NHP18 + df_bed.NumUrbanBeds_NHP18

In [ ]:
def get_area(s):
    if pd.isnull(s):
        return s
    temp = re.findall(r'\d+.?\d+', s)
    temp = [x.replace(',','') for x in temp]
    res = list(map(int, temp))
    return res

def get_density(s):
    if pd.isnull(s):
        return s
    temp = re.findall(r'\d+[.\.]?\d+', s)
    temp = [x.replace(',','') for x in temp]
    res = list(map(float, temp))
    return res

## Trend Analysis

In [ ]:
df_states = df.copy()
def add_days(df,new_col,basis):
    states = {}
    df[new_col] = 0
    for i in range(len(df_states)):
        if df_states.loc[i,'state'] in states:
            df_states.loc[i,new_col] = (df_states.loc[i,'date'] - states[df_states.loc[i,'state']]).days
        else:
            if df_states.loc[i,basis] > 0:
                states[df_states.loc[i,'state']] = df_states.loc[i,'date']
    return df
df_states = add_days(df_states,'day_since_inf','Confirmed')
df_states = add_days(df_states,'day_since_death','Deaths')
df_states = add_days(df_states,'day_since_cure','Cured')

In [ ]:
fig = px.line(df_states,x='day_since_inf',y='Confirmed',color='state',title='Cumulative cases over time')
fig.update_layout(template="simple_white")
fig.update_xaxes(title_text='Days since first infection was reported')
fig.update_yaxes(title_text='No. of Confirmed cases')
fig.show()

In [ ]:
fig = px.line(df_states,x='day_since_death',y='Deaths',color='state',title='Cumulative deaths over time')
fig.update_layout(template="simple_white")
fig.update_xaxes(title_text='Days since first death was reported')
fig.update_yaxes(title_text='No. of Confirmed deaths')
fig.show()

In [ ]:
fig = px.line(df_states,x='day_since_cure',y='Cured',color='state',title='Cumulative recoveries over time')
fig.update_layout(template="simple_white")
fig.update_xaxes(title_text='Days since first recovery was reported')
fig.update_yaxes(title_text='No. of Confirmed recoveries')
fig.show()

In [ ]:
df_states.sort_values(by=['state','date'],inplace=True)
df_states.reset_index(inplace=True,drop=True)
df_states_daily = add_daily_measures(df_states)
df_states_daily.fillna(0,inplace=True)

In [ ]:
states = df_states_daily['state'].unique().tolist()
df_roll = pd.DataFrame()
for state in states:
    df_state = df_states_daily[df_states_daily['state']==state]
    df_state['roll_avg_c'] = np.round(df_state['Daily Cases'].rolling(7).mean())
    df_state['roll_avg_d'] = np.round(df_state['Daily Deaths'].rolling(7).mean())
    df_state['roll_avg_r'] = np.round(df_state['Daily Cured'].rolling(7).mean())
    df_roll = df_roll.append(df_state,ignore_index=True)

In [ ]:
fig = px.line(df_roll,x='day_since_inf',y='roll_avg_c',color='state',title='Daily cases over time')
fig.update_layout(template="simple_white")
fig.update_xaxes(title_text='Days since first infection was reported')
fig.update_yaxes(title_text='7-Day Rolling average')
fig.show()

In [ ]:
fig = px.line(df_roll,x='day_since_inf',y='roll_avg_d',color='state',title='Daily deaths over time')
fig.update_layout(template="simple_white")
fig.update_xaxes(title_text='Days since first infection was reported')
fig.update_yaxes(title_text='7-Day Rolling average')
fig.show()

In [ ]:
fig = px.line(df_roll,x='day_since_inf',y='roll_avg_r',color='state',title='Daily recoveries over time')
fig.update_layout(template="simple_white")
fig.update_xaxes(title_text='Days since first infection was reported')
fig.update_yaxes(title_text='7-Day Rolling average')
fig.show()

In [ ]:
fig = go.Figure(data=go.Heatmap(
        z=df_states_daily['Daily Cases'],
        x=df_states_daily.date,
        y=df_states_daily.state,
        colorscale=px.colors.sequential.Blues))
fig.update_layout(height=600,template="plotly_white",title='Heatmap: Daily cases per day',yaxis_nticks=36)
fig.show()


In [ ]:
fig = go.Figure(data=go.Heatmap(
        z=df_states_daily['Daily Deaths'],
        x=df_states_daily.date,
        y=df_states_daily.state,
        colorscale=px.colors.sequential.Reds))
fig.update_layout(height=600,template="plotly_white",title='Heatmap: Daily deaths per day',yaxis_nticks=36)
fig.show()

In [ ]:
fig = go.Figure(data=go.Heatmap(
        z=df_states_daily['Daily Cured'],
        x=df_states_daily.date,
        y=df_states_daily.state,
        colorscale=px.colors.sequential.Greens))
fig.update_layout(height=600,template="plotly_white",title='Heatmap: Daily recoveries per day',yaxis_nticks=36)
fig.show()

In [ ]:
n = 5
df_curr = df_states[df_states['date']==df_states.date.max()]
states = df_curr.nlargest(n,'Confirmed')['state'].values.tolist()

In [ ]:
def plot_state(state=state):
    df_state = df_states[df_states['state']==state]
    df_state['roll_avg_c'] = df_state['Daily Cases'].rolling(7).mean()
    df_state['roll_avg_d'] = df_state['Daily Deaths'].rolling(7).mean()
    df_state['roll_avg_r'] = df_state['Daily Cured'].rolling(7).mean()
    fig = go.Figure()
    fig.add_trace(go.Bar(name='Cases',x=df_state['date'],y=df_state['Daily Cases'],marker_color='royalblue',yaxis='y'))
    fig.add_trace(go.Scatter(name='7-day rolling avg',x=df_state['date'],y=df_state['roll_avg_c'],marker_color='black',yaxis='y'))
    fig.add_trace(go.Bar(name='Deaths',x=df_state['date'],y=df_state['Daily Deaths'],marker_color='crimson',yaxis='y2'))
    fig.add_trace(go.Scatter(name='7-day rolling avg',x=df_state['date'],y=df_state['roll_avg_d'],marker_color='black',yaxis='y2'))
    fig.add_trace(go.Bar(name='Recoveries',x=df_state['date'],y=df_state['Daily Cured'],marker_color='limegreen',yaxis='y3'))
    fig.add_trace(go.Scatter(name='7-day rolling avg',x=df_state['date'],y=df_state['roll_avg_r'],marker_color='black',yaxis='y3'))
    # Update axes
    fig.update_layout(
        xaxis=dict(
        autorange=True,
        rangeslider=dict(
            autorange=True,
        ),
        type="date"
    ),
    yaxis=dict(
        anchor="x",
        autorange=True,
        domain=[0, 0.3],
        linecolor="royalblue",
        mirror=True,
        showline=True,
        side="right",
        tickfont={"color": "royalblue"},
        tickmode="auto",
        ticks="",
        title='Cases',
        titlefont={"color": "royalblue","size":10},
        type="linear",
        zeroline=False
    ),
    yaxis2=dict(
        anchor="x",
        autorange=True,
        domain=[0.35, 0.65],
        linecolor="crimson",
        mirror=True,
        showline=True,
        side="right",
        tickfont={"color": "crimson"},
        tickmode="auto",
        ticks="",
        title = 'Deaths',
        titlefont={"color": "crimson","size":10},
        type="linear",
        zeroline=False
    ),
    yaxis3=dict(
        anchor="x",
        autorange=True,
        domain=[0.7, 1],
        linecolor="limegreen",
        mirror=True,
        showline=True,
        side="right",
        tickfont={"color": "limegreen"},
        tickmode="auto",
        ticks="",
        title="Recoveries",
        titlefont={"color": "limegreen","size":10},
        type="linear",
        zeroline=False
    )
    )
    fig.update_layout(title=state,showlegend=False,template='plotly_white')
    fig.show()

In [ ]:
plot_state(states[0])

In [ ]:
plot_state(states[1])

In [ ]:
plot_state(states[2])

In [ ]:
plot_state(states[3])

In [ ]:
plot_state(states[4])

In [ ]:
import pandas as pd
import numpy as np
import datetime
import requests
import warnings

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns
import squarify
import plotly.offline as py
import plotly_express as px

from IPython.display import Image
warnings.filterwarnings('ignore')
%matplotlib inline
age_details = pd.read_csv('../input/covid19-in-india/AgeGroupDetails.csv')
india_covid_19 = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
hospital_beds = pd.read_csv('../input/covid19-in-india/HospitalBedsIndia.csv')
individual_details = pd.read_csv('../input/covid19-in-india/IndividualDetails.csv')
ICMR_labs = pd.read_csv('../input/covid19-in-india/ICMRTestingLabs.csv')
state_testing = pd.read_csv('../input/covid19-in-india/StatewiseTestingDetails.csv')
population = pd.read_csv('../input/covid19-in-india/population_india_census2011.csv')

# world_population = pd.read_csv('../input/population-by-country-2020/population_by_country_2020.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-04-2020.csv')

india_covid_19['Date'] = pd.to_datetime(india_covid_19['Date'],dayfirst = True)
state_testing['Date'] = pd.to_datetime(state_testing['Date'])


## Age Analysis

In [ ]:
age_details.head()


In [ ]:
labels = list(age_details['AgeGroup'])
sizes = list(age_details['TotalCases'])

explode = []

for i in labels:
    explode.append(0.05)
    
plt.figure(figsize= (20,15))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=9, explode =explode)
centre_circle = plt.Circle((0,0),0.70,fc='white')

fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.title('India - Age Group wise Distribution',fontsize = 20)
plt.legend()
plt.axis('equal')  
plt.savefig('agewise_distribution.png',dpi=500)
plt.tight_layout()

We could see that the **age group <40 is the most affected** which is against the trend which says elderly people are more at risk of being affected. Only 17% of people >60 are affected.

In [ ]:
labels = ['Missing', 'Male', 'Female']
sizes = []
sizes.append(individual_details['gender'].isnull().sum())
sizes.append(list(individual_details['gender'].value_counts())[0])
sizes.append(list(individual_details['gender'].value_counts())[1])

explode = (0, 0.1, 0)
colors = ['#ffcc99','#66b3ff','#ff9999']

plt.figure(figsize= (15,10))
plt.title('Percentage of Gender',fontsize = 30)
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',shadow=True, startangle=100)
plt.axis('equal')
plt.legend()
# plt.savefig('percentage_gener.png',dpi=500)
plt.tight_layout()

**80% of the patients gender information is missing. Let's analyse with remaining the data.

In [ ]:
labels = ['Male', 'Female']
sizes = []
sizes.append(list(individual_details['gender'].value_counts())[0])
sizes.append(list(individual_details['gender'].value_counts())[1])

explode = (0.1, 0)
colors = ['#66b3ff','#ff9999']

plt.figure(figsize= (15,10))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
        shadow=True, startangle=90)

plt.title('Percentage of Gender (Ignoring the Missing Values)',fontsize = 20)

plt.axis('equal')
plt.legend()
# plt.savefig("percentage_of_gender-missing.png",dpi=500)
plt.tight_layout()

Men are the most affected accounting to 67%. But, remember we have ~80% data missing.

In [ ]:
dates = list(confirmed_df.columns[4:])
dates = list(pd.to_datetime(dates))
dates_india = dates[8:]

## Comparison with Other countries

In [ ]:
countries = ['China','US', 'Italy', 'Spain', 'France','India']

global_confirmed = []
global_recovered = []
global_deaths = []
global_active = []

for country in countries:
    k =df1[df1['Country/Region'] == country].loc[:,'1/30/20':]
    global_confirmed.append(k.values.tolist()[0]) 

    k =df2[df2['Country/Region'] == country].loc[:,'1/30/20':]
    global_deaths.append(k.values.tolist()[0]) 

    k =df3[df3['Country/Region'] == country].loc[:,'1/30/20':]
    global_deaths.append(k.values.tolist()[0])  

plt.figure(figsize= (20,15))
plt.xticks(rotation = 90 ,fontsize = 20)
plt.yticks(fontsize = 30)
plt.xlabel("Dates",fontsize = 30)
plt.ylabel('Total cases',fontsize = 30)
plt.title("Comparison with other Countries" , fontsize = 20)

for i in range(len(countries)):
    plt.plot_date(y= global_confirmed[i],x= dates_india,label = countries[i],linestyle ="-")
# plt.savefig("comparison_with_countries.png",dpi=500);    
plt.legend();

### Statewise Testing and Healthcare Insights <a id="6"></a>

In [ ]:
hospital_beds =hospital_beds.drop([36])
cols_object = list(hospital_beds.columns[2:8])

for cols in cols_object:
    hospital_beds[cols] = hospital_beds[cols].astype(int,errors = 'ignore')

In [ ]:
top_10_primary = hospital_beds.nlargest(10,'NumPrimaryHealthCenters_HMIS')
top_10_community = hospital_beds.nlargest(10,'NumCommunityHealthCenters_HMIS')
top_10_district_hospitals = hospital_beds.nlargest(10,'NumDistrictHospitals_HMIS')
top_10_public_facility = hospital_beds.nlargest(10,'TotalPublicHealthFacilities_HMIS')
top_10_public_beds = hospital_beds.nlargest(10,'NumPublicBeds_HMIS')

plt.figure(figsize=(20,15))
plt.suptitle('Top 10 States in each Health Facility',fontsize=30)
plt.subplot(221)
plt.title('Primary Health Centers')
plt.barh(top_10_primary['State/UT'],top_10_primary['NumPrimaryHealthCenters_HMIS'],color ='#87479d');

plt.subplot(222)
plt.title('Community Health Centers')
plt.barh(top_10_community['State/UT'],top_10_community['NumCommunityHealthCenters_HMIS'],color = '#9370db');

plt.subplot(224)
plt.title('Total Public Health Facilities')
plt.barh(top_10_community['State/UT'],top_10_public_facility['TotalPublicHealthFacilities_HMIS'],color='#9370db');

plt.subplot(223)
plt.title('District Hospitals')
plt.barh(top_10_community['State/UT'],top_10_district_hospitals['NumDistrictHospitals_HMIS'],color = '#87479d');
plt.savefig("top10_states_in_each_health.png",dpi=500)

In [ ]:
top_rural_hos = hospital_beds.nlargest(10,'NumRuralHospitals_NHP18')
top_rural_beds = hospital_beds.nlargest(10,'NumRuralBeds_NHP18')
top_urban_hos = hospital_beds.nlargest(10,'NumUrbanHospitals_NHP18')
top_urban_beds = hospital_beds.nlargest(10,'NumUrbanBeds_NHP18')

plt.figure(figsize=(20,15))
plt.suptitle('Urban and Rural Health Facility',fontsize=20)
plt.subplot(221)
plt.title('Rural Hospitals')
plt.barh(top_rural_hos['State/UT'],top_rural_hos['NumRuralHospitals_NHP18'],color = '#87479d');

plt.subplot(222)
plt.title('Urban Hospitals')
plt.barh(top_urban_hos['State/UT'],top_urban_hos['NumUrbanHospitals_NHP18'],color = '#9370db');

plt.subplot(223)
plt.title('Rural Beds')
plt.barh(top_rural_beds['State/UT'],top_rural_beds['NumRuralBeds_NHP18'],color = '#87479d');

plt.subplot(224)
plt.title('Urban Beds')
plt.barh(top_urban_beds['State/UT'],top_urban_beds['NumUrbanBeds_NHP18'],color = '#9370db');
plt.savefig("urban_and_rural_health.png",dpi=500)


In [ ]:
state_testing.head()

# Please Upvote if you liked.